In [1]:
# Read data with intellexer spell checker
data = spark. \
       read. \
       load("/home/danilo/Documents/JSL/corpus/spell-testset1.txt",
           format="com.databricks.spark.csv",
           header="false")

In [2]:
data.show(5)

+--------------------+
|                 _c0|
+--------------------+
|contented: conten...|
| beginning: begining|
|problem: problam ...|
|      driven: dirven|
|ecstasy: exstacy ...|
+--------------------+
only showing top 5 rows



In [3]:
from pyspark.sql import functions as F

split_col = F.split(data['_c0'], ':')
data = data.withColumn('word', split_col.getItem(0))
data = data.withColumn('misspell', split_col.getItem(1))
data = data.select('word', 'misspell')

In [4]:
data.show(5, False)

+---------+----------------------------------------+
|word     |misspell                                |
+---------+----------------------------------------+
|contented| contenpted contende contended contentid|
|beginning| begining                               |
|problem  | problam proble promblem proplen        |
|driven   | dirven                                 |
|ecstasy  | exstacy ecstacy                        |
+---------+----------------------------------------+
only showing top 5 rows



In [5]:
data = data.toPandas()
data.head()

,word,misspell
0,contented,contenpted contende contended contentid
1,beginning,begining
2,problem,problam proble promblem proplen
3,driven,dirven
4,ecstasy,exstacy ecstacy


In [6]:
import pandas as pd

word_list = []
misspell_list = []
for index, row in data.iterrows():
    word, misspell = row['word'], row['misspell']
    misspell = misspell.split(' ')
    for m in misspell:
        if m != '':
            word_list.append(word)
            misspell_list.append(m)
# Creates a dataframe of misspelled words
mw_df = pd.DataFrame({"word": word_list, "misspell": misspell_list})   

In [7]:
mw_df.head()

,misspell,word
0,contenpted,contented
1,contende,contented
2,contended,contented
3,contentid,contented
4,begining,beginning


In [8]:
mw_df.to_csv("misspelled_words.csv", index=False, header=True)